Tests the reorient task by performing expected user actions.

Requires that this notebook is in the same folder as tasks.py

In [1]:
import tasks
from tasks import robj
import json
import pandas as pd
import utils
from gridfs import GridFS
pd.set_option('display.max_colwidth', 400)

In [2]:
# initial query - mdb find 
query = "india"
tid = 'a2'
res = tasks.querySearch.delay(query_string=query, teleoscope_id=tid)
res = res.get()

Lets say we end up wanting to commit the following posts:

POSITIVE:

id: j1f2rk

title: AITA for getting offended when my dad asked me to do chores?

id: j1f1jh

title: AITA for not washing the dishes?

In [3]:
res = robj.delay(teleoscope_id=tid, positive_docs=['dx17qr'], negative_docs=[], query=query)
res = res.get()

In [4]:
db = utils.connect()

In [5]:
def displayTopDocs(db, query, tid, n):
    gridfs_id = db.queries.find_one({"query": query, "teleoscope_id": tid})['ranked_post_ids']
    fs = GridFS(db, "queries")
    obj = fs.get(gridfs_id).read()
    obj = json.loads(obj)
    top_n_id = [x[0] for i,x in enumerate(obj) if i < n]
    top_n_docs = db.clean.posts.v2.find({'id': {'$in': top_n_id}}, projection={'selftext':1, 'title':1, 'id':1, '_id':0})
    top_n_docs = list(top_n_docs)
    
    df = pd.DataFrame(top_n_docs)
    
    df2 = pd.DataFrame(obj)
    df2.columns = ['id', 'cosine_similarity']
    
    df = df.merge(right=df2, on='id', how='left').copy()
    df = df.sort_values('cosine_similarity', ascending=False)
    return df

In [6]:
df = displayTopDocs(db, query, tid, 100)

In [7]:
df.head(20)

,id,selftext,title,cosine_similarity
46,dx17qr,"\n\nThis is pretty short\n\nSo I am from Pakistan, I live in Canada currentyl, but I was recently staying in Pakistan recently to meet family and relatives. When I was there, I joined a Whatsapp Group Chat with some of the members being from India. One of the Indian guys there finds out I'm Pakistani and asks me what I think about Bharat (Bharat is what Indians call India). I decide to be a l...",AITA for making a joke about India?,0.967378
45,dpdk49,"\n\nI am a US Immigrant from Pakistan. Whenever I make new friends or socialise, I tell people that I'm Indian. My relatives in the US do this frequently too and infact my uncle owns an Indian food restaurant without any references to Pakistan in it. \nI can't remember how many times I've done this with acquaintances or in situations where I know people wouldn't enquire further about my backgr...",AITA for LARPing in real life?,0.708165
70,gorsoq,"So, i was born in the UK but my parents were immigrants from India who moved for better job opportunity and better quality of life. I lived in India until i was 3 at which point i moved back to the UK and have stayed there ever since. Recently, i was talking to my parents about India and how kids are really hardworking over there and i made a joke about me being the exception to the rule. He l...",AITA for getting annoyed at my dad for not calling me Indian,0.679071
5,aw1gfk,"Okay so I'm indian but I live in and grew up in Canada. My parents actually grew up in India. \n\n&amp;#x200B;\n\nOn to the story, one girl who goes to my school posted a bunch of snaps on her story about how India is the real terrorist and how her nation(Pakistan) is 'just more mature'. I felt like it was really fucking rude and could be just because I'm indian and found it offensive but I fe...",WIBTA if I called someone out for a political rant?,0.664306
63,fqzv8e,"For reference, I’m also Indian.Apparently this is something some other Indian kids I know, but idk. Let’s see what y’all think.\n\nSo some time ago, when school was still on, I have having a conversation with some friends of mine who are white while we were jogging in gym class. The topic of dating and girls came up, and I just said my Indian parents would never be down for that. Apparently my...",AITA for trashing India as a country and being ‘colorist’?,0.660313
6,ax5s35,"i (20M) am at university in the UK. i came abroad from india dehli. My classmate is also 20M and he is from pakistan. \n\nin my Law class for the law of International Use of Force the topic of India and Pakistan conflict arose. not to get into the specific politics of this but while i was arguing with him i said:\n\n""pakistan is a shit hole country anyway. the conflict with india is bad but it...",AITA for saying Pakistan is a shithole?,0.643398
77,hchgam,"Just saying now it’s a long read. TL;DR at the end. \n\nWe’re both 25, and we’ve been together for somewhat over three years now. I love her more than anything, but this situation is proving to be a challenge. \n\nSome background about me: I’m ethnically from a India. My parents immigrated here to the United States before I was born, so I was born and raised here. I grew up in an area for some...",AITA for not wanting to vacation in my parent’s country with my girlfriend?,0.640084
61,fej2qy,Basically it started as normal I being from India doesn't like Pakistan very much but when I told her that I don't care about pakistan she started accusing of me being a robot and not caring about the human life . I told her that Pakistan is not good country towards India historically she is like it doesn't matter you should not hate and it made me angry and we had a huge fight over it . She a...,AITA For Not being politically correct with my girlfriend.,0.634155
7,ayd0yx,"So the other day i was in school and my friend said something about India (he’s Indian) that was a racist joke about Indian beliefs. Another class mate looked offended after s

Now lets say we want to negatively commit this post:

id: hey6i9

selftext: New to reddit and on mobile\nMe 23F\nFlatmate 24M\n\n\nSo, background - I have a lot of anxiety, which is worsened by kitchen uncleanliness. If there is a sink piled with dishes, or if there’s dirty dishes throughout the kitchen, I can’t wash my own stuff without panicking, and if things get bad I literally live on toast which I eat in my bedroom, and hide my dirty dishes in my bedroom since I...

(basically it mentiones flatmate, but we only want to see family stuff)

In [ ]:
res = robj.delay(teleoscope_id=tid, positive_docs=[], negative_docs=['hey6i9'], query=query)
res = res.get()

In [ ]:
df = displayTopDocs(db, query, tid, 100)

In [ ]:
df.tail(20)

Now lets say we want to explore the topic of divorce:

So we positively commit this doc:

id: apmqtk

selftext: Ok so I’m gonna lay some ground work, so even before my parents got divorced I was always the kid who ended up helping clean the kitchen after dinner the most but since my parents have been separated since September 2017 my dad sorta treats me like a housewife and not a newly 14 year old girl. For background, I live with 4 other people, my dad my 2 sisters (18, and 20) and my younger brother

In [ ]:
res = robj.delay(teleoscope_id=tid, positive_docs=['apmqtk'], negative_docs=[], query=query)
res = res.get()
df = displayTopDocs(db, query, tid, 100)
df.head(20)

Now lets say we want to explore the topic of divorce:

So we positively commit this doc:

id: e4pvkp

selftext: I’m 19F and my bio brother is 14. I currently live with my boyfriend. My brother lives with my mom, and our half brother who’s 5. Littlest brother was unplanned. My mom is divorced from both of our dads.\n\nWhen I lived at home I stood in as a second parent. I did all the cleaning, cooking, errands, and picked up my brothers from school every day. So much so that I was unable to have a normal

In [ ]:
res = robj.delay(teleoscope_id=tid, positive_docs=['e4pvkp'], negative_docs=[], query=query)
res = res.get()
df = displayTopDocs(db, query, tid, 100)
df.head(20)